<a href="https://colab.research.google.com/github/AstronautCharlie/NLP/blob/master/Copy_of_PS1_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

In [63]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from nltk import word_tokenize 
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
import numpy as np

class LemmaTokenizer(object): 
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, data):
    return [self.wnl.lemmatize(t) for t in word_tokenize(data)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):

  # Get a list of the reviews as strings 
  reviews = [corpus[i]['text'] for i in range(len(corpus))]
  
  # Get matrix of word counts 
  vectorizer = CountVectorizer(token_pattern = '[a-zA-Z]+', analyzer='word')
  texts = vectorizer.fit_transform(reviews).toarray()

  # Get the classes of each review
  classes = [corpus[i]['class'] for i in range(len(corpus))]

  # Get the features used to train the model, which in this case is just 
  # the list of words in the model 
  vocab = vectorizer.get_feature_names()

  return texts,classes,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):

  # Parse corpus to get reviews 
  reviews = [corpus[i]['text'] for i in range(len(corpus))]
  
  # Get correct class of each review 
  classes = [corpus[i]['class'] for i in range(len(corpus))] 

  # Create matrix for basic features 
  basic_vectorizer = CountVectorizer(analyzer='word',
                                     token_pattern='[a-zA-Z]+',
                                     ngram_range=(1,2),
                                     max_df=0.8,
                                     )
  texts_basic = basic_vectorizer.fit_transform(reviews)
  vocab_basic = basic_vectorizer.get_feature_names()

  # Create TFIDF matrix
  tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                     token_pattern='[a-zA-Z]+',
                                     ngram_range=(1,2),
                                     max_df=0.8,
                                     )
  text_tfidf = tfidf_vectorizer.fit_transform(reviews)
  vocab_tfidf = tfidf_vectorizer.get_feature_names()

  # Combine feature matrices 
  texts = scipy.sparse.hstack((texts_basic, text_tfidf))
  #texts = texts_basic

  # Combine vocabs 
  vocab = vocab_basic + vocab_tfidf
  #vocab = vocab_basic

  return texts,classes,vocab

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [0]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [0]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.826000
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'perfectly', 'edge', 'enjoyable', 'sherri', 'masterpiece', 'excellent', 'gas', 'overall', 'using', 'quite', 'fun', 'follows', 'pulp', 'solid', 'holds', 'seen', 'loved']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'unfortunately', 'cheap', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'looks', 'maybe', 'reason']
----------L2 Norm-----------
The model's average accuracy is 0.832000
The most informative terms for pos are: ['fun', 'back', 'great', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'memorable', 'terrific', 'yet', 'pulp', 'job', 'true', 'performances', 'very', 'different', 'solid']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'only', 'script', 'boring', 'awful', 'plot

Run the following to train and evaluate two models using extended features:

In [112]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.833500
The most informative terms for pos are: ['even if', 'flaws', 'memorable', 'due to', 'terrific', 'overall', 'others', 'master', 'gas', 'hilarious', 'masterpiece', 'perfectly', 'as much', 'by a', 'great', 'works', 'fun', 'follows', 'filmmaking', 'portrayed']
The most informative terms for neg are: ['waste', 'awful', 'headed', 'unfortunately', 'mess', 'ridiculous', 'should have', 'boring', 'worst', 'poor', 'cheap', 'lame', 'write', 'bad', 'designed', 'supposed', 'about it', 'would have', 'metro', 'nowhere']
----------L2 Norm-----------
The model's average accuracy is 0.862000
The most informative terms for pos are: ['well', 'great', 'fun', 'he is', 'also', 'back', 'very', 'seen', 'life', 'many', 'people', 'quite', 'american', 'most', 'yet', 'by a', 'while', 'others', 'job', 'true']
The most informative terms for neg are: ['bad', 'unfortunately', 'plot', 'only', 'worst', 'nothing', 'any', 'waste', 'boring', 'poor', 'scri